In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/Users/aa/Desktop/coding/p3ai/t2final/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading AI model...


Loading weights: 100%|██████████| 76/76 [00:00<00:00, 1353.71it/s, Materializing param=transformer.wte.weight]            
GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [31]:
import torch
import torch.nn.functional as F

# 1. The Prompt: Primed for Abstract Cosmic Horror/Beauty
# We end with a comma to encourage continuation rather than a full stop.
txt = "Upon the lip of the singularity, where light screams into silence and gravity drinks the dust of dead stars,"

print(f"Start prompt: '{txt}'")
print("Generating...\n")

# 2. Tuning for "Beautiful/Abstract" Generation
# Temp 1.0 + Top-P 0.92 is the "Goldilocks" zone for poetry.
# It allows for rare words (high temp) but cuts off nonsense (top-p).
temp = 1.0        # Allows for creative/rare word choices
top_p = 0.92      # "Nucleus" threshold: cuts off the tail of garbage tokens
rep_penalty = 1.2 # Stronger penalty prevents "the the of the" loops
nlns = 4          # Number of lines
wpln = 5          # Min words per line
wplnmax = 14      # Max words per line

# 3. Setup
lns = []
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initial encoding
input_ids = tokenizer.encode(txt, return_tensors="pt").to(device)
current_ids = input_ids

# KV-Caching Variable (The Speed Optimization)
# This stores the "memory" of previous tokens so we don't re-process the whole poem every loop.
past_key_values = None 

# Pre-calculate forbidden tokens (Newlines/EOS) to stop early breaking
newline_tokens = tokenizer.encode("\n", add_special_tokens=False)
eos_token = tokenizer.eos_token_id

print(f"--- Poem Start ---\n{txt}")

for i in range(nlns):
    lntxt_ids = [] # Store token IDs for the current line
    comp = False
    
    while not comp:
        with torch.no_grad():
            # OPTIMIZATION: Only feed the *last* token if we have cache
            if past_key_values is None:
                outputs = model(current_ids, use_cache=True)
            else:
                outputs = model(current_ids[:, -1:], past_key_values=past_key_values, use_cache=True)
            
            logits = outputs.logits[:, -1, :]
            past_key_values = outputs.past_key_values # Update cache

        # A. Apply Repetition Penalty (prevent looping)
        # We penalize all tokens currently in the context
        for t_id in set(input_ids[0].tolist() + lntxt_ids):
            # If logit is positive, divide; if negative, multiply (to make it smaller/more negative)
            if logits[0, t_id] < 0:
                logits[0, t_id] *= rep_penalty
            else:
                logits[0, t_id] /= rep_penalty

        # B. Apply Temperature
        logits = logits / temp

        # C. Ban undesirable tokens (EOS and early newlines)
        logits[0, eos_token] = -float("inf")
        for nl in newline_tokens:
            logits[0, nl] = -float("inf")

        # D. Nucleus (Top-P) Sampling
        probs = F.softmax(logits, dim=-1)
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

        # Create mask for tokens to remove
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0 # Always keep at least one token

        # Scatter the mask back to original indices
        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        logits[indices_to_remove] = -float("inf")

        # E. Final Sample
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        # Update trackers
        current_ids = torch.cat([current_ids, next_token], dim=-1) # Just for full context record
        lntxt_ids.append(next_token.item())
        
        # F. Smart Stopping Logic
        # Decode only the current line to check length/punctuation
        line_text = tokenizer.decode(lntxt_ids)
        words = line_text.strip().split()
        word_count = len(words)

        # Check for punctuation end
        is_end_punct = line_text.strip() and line_text.strip()[-1] in ".?!;"

        if word_count >= wplnmax:
            comp = True
        elif word_count >= wpln and is_end_punct:
            comp = True

    # Formatting and storage
    final_line = tokenizer.decode(lntxt_ids).strip()
    lns.append(final_line)
    
    # Update input_ids for the next line's repetition penalty context
    input_ids = torch.cat([input_ids, torch.tensor([lntxt_ids]).to(device)], dim=-1)
    
    print(f"Line {i+1}: {final_line}")

print("\n--- Final Composition ---")
print(txt)
for line in lns:
    print(line)

Start prompt: 'Upon the lip of the singularity, where light screams into silence and gravity drinks the dust of dead stars,'
Generating...

--- Poem Start ---
Upon the lip of the singularity, where light screams into silence and gravity drinks the dust of dead stars,
Line 1: it is evident that this noise has been echoed many times since—a manifestation not
Line 2: only on Earth itself, but in its occupants as well.
Line 3: True to music; however bitter they might be for anyone contemplating a green lost
Line 4: cause: an abundant selection with depressingly unpleasant lyrics starring Audre Lorde adorning Lazer drum

--- Final Composition ---
Upon the lip of the singularity, where light screams into silence and gravity drinks the dust of dead stars,
it is evident that this noise has been echoed many times since—a manifestation not
only on Earth itself, but in its occupants as well.
True to music; however bitter they might be for anyone contemplating a green lost
cause: an abundant selec